In [ ]:
from pandas import read_excel


df = (
    read_excel("../input/thai_exams_translated.xlsx")
    .query("skip_flag != 'Y'")
)

In [3]:
df.shape

(583, 8)

In [4]:
df.head(2)

,ID,en_question,en_a,en_b,en_c,en_d,en_e,skip_flag
0,a_level_001,Which of the following statements is incorrect...,Light behaves both as a wave and a particle.,"When light passes through a double slit, it un...",Natural sunlight is unpolarized light.,A rainbow occurs solely due to the phenomenon ...,"When light travels through different mediums, ...",NaN
1,a_level_002,Which of the following correctly describes soc...,Each society may or may not have social approval.,Social approval through persuasion occurs when...,Government agencies will become involved in ma...,"Mockery, gossip, fines, imprisonment are socia...",Social sanctioning through punishment can be b...,NaN


In [5]:
keep_idx = df.index

In [ ]:
df_th = (
    read_excel("../input/data_gen_scripts/data/thai_exams.xlsx")
    .iloc[keep_idx, :]
)

In [14]:
df_th.shape

(583, 13)

In [16]:
df_th.head(2)

,question,a,b,c,d,e,answer,subject,split,ID,exam,year,no
0,ข้อใดกล่าวไม่ถูกต้องเกี่ยวกับแสง,แสงประพฤติตัวได้ทั้งแบบคลื่นและอนุภาค,เมื่อแสงผ่านสลิตคู่จะเกิดการแทรกสอดกันท าให้เก...,แสงอาทิตย์ธรรมชาติเป็นแสงไม่โพลาไรซ์,รุ้งกินน้ำเกิดขึ้นจากปรากฎการณ์หักเหของแสงเพีย...,เมื่อแสงเคลื่อนที่ผ่านตัวกลางที่ต่างกันจะมีควา...,d,science,train,a_level_001,a_level,NaN,NaN
1,ข้อใดกล่าวถึงสิทธานุมัติทางสังคมได้ถูกต้อง,แต่ละสังคมจะมีหรือไม่มีสิทธานุมัติทางสังคมก็ได้,สิทธานุมัติทางสังคมโดยการจูงใจ กระทำเมื่อสมาชิ...,หน่วยงานของรัฐจะเข้ามาเกี่ยวข้องในเรื่องสิทธาน...,การเยาะเย้ยถากถาง นินทา ปรับเงิน จำคุก เป็นสิท...,สิทธานุมัติทางสังคมโดยการลงโทษมีทั้งที่เป็นทาง...,e,social,train,a_level_002,a_level,NaN,NaN


In [17]:
df = (
    df
    .merge(
        df_th[["ID", "answer", "subject", "split"]],
        on="ID",
        how="left",
    )
)

In [42]:
import numpy as np

def prepare_question(question, a, b, c, d, e):
    q = f"{question}\n\nA. {a}\n\nB. {b}\n\nC. {c}\n\nD. {d}"
    if e is not np.nan:
        q += f"\n\nE. {e}"
    return q

In [43]:
df["question"] = df[["en_question", "en_a", "en_b", "en_c", "en_d", "en_e"]].apply(
    lambda x: prepare_question(
        x["en_question"], 
        x["en_a"],
        x["en_b"],
        x["en_c"],
        x["en_d"],
        x["en_e"],
    ),
    axis=1
)
df["answer"] = df["answer"].str.upper()

In [38]:
df[["ID", "question", "answer"]].head(2)

,ID,question,answer
0,a_level_001,Which of the following statements is incorrect...,D
1,a_level_002,Which of the following correctly describes soc...,E


In [39]:
print(df.loc[0, "question"])

Which of the following statements is incorrect about light?

A. Light behaves both as a wave and a particle.

B. When light passes through a double slit, it undergoes interference, creating fringes on the screen.

C. Natural sunlight is unpolarized light.

D. A rainbow occurs solely due to the phenomenon of light refraction.

E. When light travels through different mediums, it will have different speeds.


In [44]:
print(df.loc[df[df["en_e"].isnull()].index[0], "question"])

Which of the following expresses air temperature in terms of the speed of sound?

A. $ t = \ frac {a-1,052} {1.08} $

B. $ t = \ frac {a+1,052} {1.08} $

C. $ t = \ frac {1,052-a} {1.08} $

D. $ t = \ frac {1.08} {a+1,052} $


In [ ]:
df[["ID", "question", "answer"]].to_json("../output/THAI_EXAM/THAI_EXAM_questions_gt_en.jsonl")

Thai

In [46]:
df_th["question"] = df_th[["question", "a", "b", "c", "d", "e"]].apply(
    lambda x: prepare_question(
        x["question"], 
        x["a"],
        x["b"],
        x["c"],
        x["d"],
        x["e"],
    ),
    axis=1
)
df_th["answer"] = df_th["answer"].str.upper()

In [47]:
df_th[["ID", "question", "answer"]].head(2)

,ID,question,answer
0,a_level_001,ข้อใดกล่าวไม่ถูกต้องเกี่ยวกับแสง\n\nA. แสงประพ...,D
1,a_level_002,ข้อใดกล่าวถึงสิทธานุมัติทางสังคมได้ถูกต้อง\n\n...,E


In [48]:
print(df_th.loc[0, "question"])

ข้อใดกล่าวไม่ถูกต้องเกี่ยวกับแสง

A. แสงประพฤติตัวได้ทั้งแบบคลื่นและอนุภาค

B. เมื่อแสงผ่านสลิตคู่จะเกิดการแทรกสอดกันท าให้เกิดริ้วบนฉาก

C. แสงอาทิตย์ธรรมชาติเป็นแสงไม่โพลาไรซ์

D. รุ้งกินน้ำเกิดขึ้นจากปรากฎการณ์หักเหของแสงเพียงอย่างเดียว

E. เมื่อแสงเคลื่อนที่ผ่านตัวกลางที่ต่างกันจะมีความเร็วไม่เท่ากัน


In [49]:
print(df_th.loc[df_th[df_th["e"].isnull()].index[0], "question"])

ข้อใดต่อไปนี้เป็นการแสดงออกถึงอุณหภูมิอากาศในแง่ของความเร็วของคลื่นเสียง

A. $ t = \ frac {a-1,052} {1.08} $

B. $ t = \ frac {a+1,052} {1.08} $

C. $ t = \ frac {1,052-a} {1.08} $

D. $ t = \ frac {1.08} {a+1,052} $


In [ ]:
df_th[["ID", "question", "answer"]].to_json("../output/THAI_EXAM/THAI_EXAM_questions_gt_th.jsonl")